# Risk Analysis

#### Risk analysis is a systematic process used by organizations to identify, assess, and mitigate potential threats and uncertainties that could impact their objectives. It involves evaluating the likelihood and potential consequences of various risks, such as financial, operational, or strategic, to make informed decisions about risk management strategies. By quantifying and prioritizing risks, businesses can allocate resources effectively, implement preventive measures, and develop contingency plans, ultimately enhancing their ability to navigate uncertainties and achieve their goals while minimizing potential losses.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# yahoo finance is used to fetch data 
import yfinance as yf
yf.pdr_override()

In [2]:
# input
# Technology Companies
symbols = ['AMD','NVDA'] 
market = ['^GSPC']
start = '2018-01-01'
end = '2023-08-01'

In [3]:
df = yf.download(symbols, start, end)
benchmark = yf.download(market, start, end)

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
# Resample the data to monthly frequency
df_monthly = df['Adj Close'].resample('M').ffill()
benchmark_monthly = benchmark['Adj Close'].resample('M').ffill()

# Calculate daily returns
returns = df_monthly.pct_change().fillna(0)

In [5]:
returns.head()

,AMD,NVDA
Date,,
2018-01-31,0.000000,0.000000
2018-02-28,-0.118632,-0.014848
2018-03-31,-0.170107,-0.043016
2018-04-30,0.082587,-0.028887
2018-05-31,0.261948,0.122036


In [6]:
returns.tail()

,AMD,NVDA
Date,,
2023-03-31,0.247264,0.196662
2023-04-30,-0.088154,-0.001008
2023-05-31,0.322703,0.363437
2023-06-30,-0.036376,0.118211
2023-07-31,0.004302,0.104652


In [7]:
def calculate_drawdowns(returns):
    wealth_index = (1 + returns).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    return drawdowns

Drawdowns = calculate_drawdowns(returns)

In [8]:
Drawdowns.head()

,AMD,NVDA
Date,,
2018-01-31,0.000000,0.000000
2018-02-28,-0.118632,-0.014848
2018-03-31,-0.268559,-0.057226
2018-04-30,-0.208151,-0.084460
2018-05-31,-0.000728,0.000000


In [9]:
average_drawdown = Drawdowns.mean()
min_drawdown = Drawdowns.min()
max_drawdown = Drawdowns.max()
standard_error = returns.std() / np.sqrt(len(df))
standard_deviation = returns.std()
confidence_level = 0.95
alpha = 1 - confidence_level

# Calculate VaR
var = returns.quantile(alpha)

# Calculate Conditional Value at Risk (CVaR)
cvar = returns[returns <= var].mean()

initial_investment = 5000 # Replace with your desired initial investment amount
var_amount = initial_investment * var

In [10]:
# Print the results for each stock
for symbol in symbols:
    print(f"Stock Symbol: {symbol}")
    print(f"Average Drawdown: {average_drawdown[symbol]:.4f}")
    print(f"Min Drawdown: {min_drawdown[symbol]:.4f}")
    print(f"Max Drawdown: {max_drawdown[symbol]:.4f}")
    print(f"Standard Error: {standard_error[symbol]:.4f}")
    print(f"Standard Deviation: {standard_deviation[symbol]:.4f}")
    print(f"VaR ({confidence_level*100}% Confidence): {var[symbol]*100:.4f}%")
    print(f"CVaR ({confidence_level*100}% Confidence): {cvar[symbol]*100:.4f}%")
    print(f"VaR Amount for {initial_investment} USD: {var_amount[symbol]:.2f} USD")
    print("\n")

Stock Symbol: AMD
Average Drawdown: -0.1755
Min Drawdown: -0.6208
Max Drawdown: 0.0000
Standard Error: 0.0045
Standard Deviation: 0.1682
VaR (95.0% Confidence): -21.4311%
CVaR (95.0% Confidence): -28.2763%
VaR Amount for 5000 USD: -1071.55 USD


Stock Symbol: NVDA
Average Drawdown: -0.1907
Min Drawdown: -0.6282
Max Drawdown: 0.0000
Standard Error: 0.0039
Standard Deviation: 0.1464
VaR (95.0% Confidence): -21.5495%
CVaR (95.0% Confidence): -26.1210%
VaR Amount for 5000 USD: -1077.48 USD


